In [1]:
import json
import math
import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import OrderedDict

In [2]:
BSM_SIZE = 101347646 # Already counted
N_FEATURES = 22
root_dir = '/home/jovyan/work/VeReMi/'
filenames = glob.glob(root_dir + '**/traceJSON*.json', recursive=True)

In [3]:
# # Count the number of BSM messages in the dataset
# line_num = 0
# for filepath in tqdm(filenames):
#     f = open(filepath)
#     for line in f:
#         if '"type":3' in line:
#             line_num += 1
    
# line_num

In [4]:
attacker_sender_map = {}

datasets = OrderedDict()

for filepath in filenames:
    path_parts = filepath.split('/')
    filename = path_parts[-1]
    dataset_name = path_parts[-3]
    _, vehicle, _, attacker = filename.split('-')[:4]
    attacker_type = int(attacker[1:])
    vehicle_id = int(vehicle)
    attacker_sender_map[f'{dataset_name}:{vehicle_id}'] = attacker_type
    datasets[dataset_name] = 1

for i, k in enumerate(datasets.keys()):
    datasets[k] = i

datasets

OrderedDict([('ConstSpeed_0709', 0),
             ('EventualStop_1416', 1),
             ('DataReplaySybil_0709', 2),
             ('RandomSpeed_1416', 3),
             ('DoSRandomSybil_1416', 4),
             ('DoSRandom_0709', 5),
             ('DataReplay_0709', 6),
             ('DataReplay_1416', 7),
             ('RandomPos_1416', 8),
             ('ConstSpeed_1416', 9),
             ('RandomPos_0709', 10),
             ('GridSybil_0709', 11),
             ('ConstPos_1416', 12),
             ('RandomPosOffset_1416', 13),
             ('DoS_0709', 14),
             ('ConstPosOffset_1416', 15),
             ('Disruptive_1416', 16),
             ('DoS_1416', 17),
             ('DelayedMessages_0709', 18),
             ('ConstSpeedOffset_0709', 19),
             ('DoSDisruptiveSybil_0709', 20),
             ('ConstPos_0709', 21),
             ('MixAll_0024', 22),
             ('DoSRandom_1416', 23),
             ('RandomSpeed_0709', 24),
             ('RandomSpeedOffset_1416', 25),
 

In [5]:
veremi = np.memmap('veremi.memmap', dtype='float32', mode='w+', shape=(BSM_SIZE, N_FEATURES))
# filenames = filenames[:1000]

veremi_pos = 0
for filepath in (pbar := tqdm(filenames)):
    f = open(filepath)
    path_parts = filepath.split('/')
    filename = path_parts[-1]
    dataset_name = path_parts[-3]
    receiver = int(filename.split('-')[1])
    dataset_id = datasets[dataset_name]
    pbar.set_description("Vehicle ID {}".format(receiver))
    gps = None
    for line in f:
        data = json.loads(line)
        if data['type'] == 2:
            gps = data
        elif data['type'] == 3:
            attack_type = attacker_sender_map.get(f'{dataset_name}:{data["sender"]}', None)
            if attack_type is not None:
                out = (
                    dataset_id,
                    data['messageID'],
                    data['rcvTime'],
                    receiver,
                    data['sender'],
                    attack_type,
                    data['pos'][0],
                    data['pos'][1],
                    data['spd'][0],
                    data['spd'][1],
                    data['acl'][0],
                    data['acl'][1],
                    data['hed'][0],
                    data['hed'][1],
                    gps['pos'][0],
                    gps['pos'][1],
                    gps['spd'][0],
                    gps['spd'][1],
                    gps['acl'][0],
                    gps['acl'][1],
                    gps['hed'][0],
                    gps['hed'][1],
                )
                veremi[veremi_pos] = out
                veremi_pos += 1

  0%|          | 0/134082 [00:00<?, ?it/s]

In [5]:
veremi = np.memmap('veremi.memmap', dtype='float32', mode='r', shape=(BSM_SIZE, N_FEATURES))

df = pd.DataFrame(veremi, columns=[
    'dataset_id',
    'message_id',
    'bsm_rcv_time',
    'receiver',
    'sender',
    'attack_type',
    'snd_pos_x',
    'snd_pos_y',
    'snd_spd_x',
    'snd_spd_y',
    'snd_acl_x',
    'snd_acl_y',
    'snd_hed_x',
    'snd_hed_y',
    'rcv_pos_x',
    'rcv_pos_y',
    'rcv_spd_x',
    'rcv_spd_y',
    'rcv_acl_x',
    'rcv_acl_y',
    'rcv_hed_x',
    'rcv_hed_y',
])

df = df[df.dataset_id == 22][:5000000]
# df.drop(df[df.attack_type.between(1, 9)].index, inplace=True) # drop faults (1 - 9)
df.drop(df[df.receiver == df.sender].index, inplace=True) # drop bad registers

df.sort_values(['dataset_id', 'sender', 'receiver', 'bsm_rcv_time'], inplace=True)
df.reset_index(inplace=True, drop=True)

# Calc delta_time based on the diff between the messages
def calc_delta(x):
    return pd.Series(np.insert(np.diff(x.bsm_rcv_time), 0, 0))

grouped = df[['dataset_id',
              'sender',
              'receiver',
              'bsm_rcv_time']].groupby(['dataset_id',
                                        'sender',
                                        'receiver'])

indexes = grouped.size().index

for k in tqdm(indexes):
    t = grouped.get_group(k)
    t = t.sort_values('bsm_rcv_time')
    delta_time = calc_delta(t)
    df.loc[t.index, 'delta_time'] = delta_time.values

df.to_feather('out_veremi/veremi.feather')
df.head(50)

  0%|          | 0/199984 [00:00<?, ?it/s]

,dataset_id,message_id,bsm_rcv_time,receiver,sender,attack_type,snd_pos_x,snd_pos_y,snd_spd_x,snd_spd_y,...,snd_hed_y,rcv_pos_x,rcv_pos_y,rcv_spd_x,rcv_spd_y,rcv_acl_x,rcv_acl_y,rcv_hed_x,rcv_hed_y,delta_time
0,22.0,294751.0,3788.500977,315.0,309.0,0.0,161.013931,839.456787,1.646781,-12.663773,...,-0.997445,206.989929,425.276154,-2.583445,14.838143,0.024616,-0.141279,-0.145191,0.989404,0.0
1,22.0,294875.0,3789.500977,315.0,309.0,0.0,162.901260,826.825500,1.635519,-12.577155,...,-0.997445,204.372025,440.204376,-2.589477,14.872789,0.009020,-0.051704,-0.145191,0.989404,1.0
2,22.0,294919.0,3790.500977,315.0,309.0,0.0,164.793976,814.265076,2.066530,-12.621676,...,-0.994547,201.924576,455.148193,-2.382418,14.977921,0.019117,-0.120086,-0.129824,0.991537,1.0
3,22.0,294963.0,3791.500977,315.0,309.0,0.0,166.627533,801.708008,1.799329,-12.614917,...,-0.996484,199.639175,470.087738,-2.426743,14.923635,-0.100315,0.617001,-0.133252,0.991082,1.0
4,22.0,295007.0,3792.500977,315.0,309.0,0.0,168.427139,789.101379,1.878291,-12.626040,...,-0.995964,196.779129,485.003326,-2.444910,14.927145,0.052619,-0.321163,-0.131786,0.991278,1.0
5,22.0,295051.0,3793.500977,315.0,309.0,0.0,170.156082,776.501892,1.800283,-12.546853,...,-0.996095,193.838211,499.839722,-2.556366,14.929475,-0.057085,0.333474,-0.138199,0.990405,1.0
6,22.0,295095.0,3794.500977,315.0,309.0,0.0,172.039856,763.885803,1.845869,-12.616168,...,-0.995847,191.667664,514.802307,-2.214218,15.022190,-0.106422,0.722111,-0.115146,0.993349,1.0
7,22.0,295139.0,3795.500977,315.0,309.0,0.0,173.789917,751.306885,1.835685,-12.546522,...,-0.995847,189.993240,527.515625,-1.614934,10.715025,0.670682,-4.449724,-0.118373,0.992969,1.0
8,22.0,295183.0,3796.500977,315.0,309.0,0.0,175.511642,738.749329,1.849733,-12.642180,...,-0.995847,188.730499,535.789124,-0.944270,6.265282,0.670685,-4.449721,-0.118373,0.992969,1.0
9,22.0,295227.0,3797.500977,315.0,309.0,0.0,177.383163,726.164490,1.838713,-12.566395,...,-0.995847,187.958725,539.616821,-0.275671,1.829243,0.670687,-4.449719,-0.118410,0.992965,1.0


In [6]:
df = pd.read_feather('out_veremi/veremi.feather')

random_state = 42

# group by sender and receiver
# TODO: group only by sender
grouped = df.groupby(['dataset_id', 'sender', 'receiver', 'attack_type'])
interactions = grouped.size().rename('seq_len').reset_index()

train = interactions.sample(frac=0.8, random_state=random_state)
test = interactions.drop(train.index)

train_full_genuine = train[train.attack_type == 0]

sample_size_5_pctg = round(len(train_full_genuine) / 0.95 - len(train_full_genuine))
train_5_pctg_atk = train[train.attack_type != 0].sample(sample_size_5_pctg, random_state=random_state)
train_95_genuine = pd.concat([train_full_genuine, train_5_pctg_atk]).sample(frac=1, random_state=random_state) # concat and shuffle

sample_size_10_pctg = round(len(train_full_genuine) / 0.9 - len(train_full_genuine))
train_10_pctg_atk = train[train.attack_type != 0].sample(sample_size_10_pctg, random_state=random_state)
train_90_genuine = pd.concat([train_full_genuine, train_10_pctg_atk]).sample(frac=1, random_state=random_state) # concat and shuffle

del train_10_pctg_atk
del train_5_pctg_atk
del interactions
del grouped

print(f'Train: {len(train)}, Train 0%: {len(train_full_genuine)}, Train 5%: {len(train_95_genuine)}, Train 10%: {len(train_90_genuine)}, Test: {len(test)}')

Train: 159987, Train 0%: 111618, Train 5%: 117493, Train 10%: 124020, Test: 39997


In [7]:
cols = ['dataset_id', 'sender', 'receiver', 'attack_type']

test_df = df.join(test.set_index(cols), how='inner', on=cols)
test_df.reset_index(drop=True).to_feather('out_veremi/test.feather')
del test_df
del test

train_df = df.join(train.set_index(cols), how='inner', on=cols)
train_df.reset_index(drop=True).to_feather('out_veremi/train.feather')
del train_df
del train

train_full_genuine_df = df.join(train_full_genuine.set_index(cols), how='inner', on=cols)
train_full_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_full_genuine.feather')
del train_full_genuine_df
del train_full_genuine

train_95_genuine_df = df.join(train_95_genuine.set_index(cols), how='inner', on=cols)
train_95_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_95_genuine.feather')
del train_95_genuine_df
del train_95_genuine

train_90_genuine_df = df.join(train_90_genuine.set_index(cols), how='inner', on=cols)
train_90_genuine_df.reset_index(drop=True).to_feather('out_veremi/train_90_genuine.feather')
del train_90_genuine_df
del train_90_genuine